In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

     |████████████████████████████████| 1.8MB 3.6MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 262.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 186.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of fosforml to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 158.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 262.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 236.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 223.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 kB 358.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 253.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 316.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 193.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 230.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 220.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 3.1 MB/s eta 0:00:00ta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 300.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 351.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 255.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 124.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.1/931.1 kB 276.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 330.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 284.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 95.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━ 121.1/190.9 MB 9.6 MB/s eta 0:00:08m

In [3]:

from fosforio import snowflake
from fosforio import get_dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#import seaborn as sns
#from imblearn.over_sampling import SMOTE
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib

In [5]:
df = pd.read_csv("HR_Analytical_Data_Attrition.csv")
df

,SALARY,SENIORITY,TENURE MONTHS,MONTHS AFTER COLLEGE,BIRTH YEAR,AGE,MAPPED ROLE CLEAN,SEX,ETHNICITY,HOSPITAL TYPE,...,OVER TIME HOURS,CHURN,BUSINESS TRAVEL,ENVIRONMENT SATISFACTION,JOB SATISFACTION,MARTIAL STATUS,PERCENTAGE SALARY HIKE,PERFORMANCE RATING,RELATIONSHIP SATISFACTION,WORK LIFE BALANCE
0,68271.546,3,8,116,1984,40,occupational,F,Black,Childrens,...,15,False,Travel_Rarely,High,Low,Married,11,Excellent,Medium,Best
1,70471.235,3,37,18,1992,32,nurse,F,Hispanic,Psychiatric,...,10,True,Travel_Rarely,High,Medium,Married,11,Outstanding,Low,Good
2,31657.073,1,13,161,1986,38,occupational,F,Black,Psychiatric,...,7,True,Travel_Rarely,High,Medium,Married,18,Excellent,Medium,Better
3,26898.652,1,57,3,1981,43,technologist,M,API,Acute Care Hospitals,...,14,True,Travel_Rarely,Very High,Medium,Divorced,12,Low,Very High,Good
4,53490.238,2,47,16,1988,36,emt,F,White,Psychiatric,...,14,False,Travel_Rarely,Very High,Low,Single,19,Good,Low,Bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,79047.028,5,106,164,1975,49,technologist,M,Hispanic,Acute Care - Department of Defense,...,6,True,Travel_Rarely,Low,Medium,Divorced,14,Excellent,Medium,Best
299996,51505.375,1,12,69,1985,39,social,F,Multiple,Acute Care - Department of Defense,...,9,False,Travel_Rarely,Low,Medium,Married,18,Good,Very High,Best
299997,97520.177,5,45,113,1969,55,physician,M,Hispanic,Acute Care Hospitals,...,1,False,Travel_Rarely,High,High,Divorced,20,Outstanding,Low,Best
299998,55666.168,2,32,5,1987,37,emt,M,White,Childrens,...,9,True,Travel_Rarely,Medium,Low,Married,23,Low,Very High,Best


In [9]:
print(df.isnull().sum())

SALARY                           0
SENIORITY                        0
TENURE MONTHS                    0
MONTHS AFTER COLLEGE             0
BIRTH YEAR                       0
AGE                              0
MAPPED ROLE CLEAN                0
SEX                              0
ETHNICITY                        0
HOSPITAL TYPE                    0
HOSPITAL OWNERSHIP               0
COMPANY NAME                     0
CITY                             0
 STATE                           0
DISTANCE                         0
DEGREE CLEAN                     0
SCHOOL END DATE                  0
JOB START DATE                   0
JOB END DATE                 90102
USER ID                          0
OVER TIME HOURS                  0
CHURN                            0
BUSINESS TRAVEL                  0
ENVIRONMENT SATISFACTION         0
JOB SATISFACTION                 0
MARTIAL STATUS                   0
PERCENTAGE SALARY HIKE           0
PERFORMANCE RATING               0
RELATIONSHIP SATISFA

In [15]:
df = df.dropna()

In [16]:
print(df.isnull().sum())

SALARY                       0
SENIORITY                    0
TENURE MONTHS                0
MONTHS AFTER COLLEGE         0
BIRTH YEAR                   0
AGE                          0
MAPPED ROLE CLEAN            0
SEX                          0
ETHNICITY                    0
HOSPITAL TYPE                0
HOSPITAL OWNERSHIP           0
COMPANY NAME                 0
CITY                         0
 STATE                       0
DISTANCE                     0
DEGREE CLEAN                 0
SCHOOL END DATE              0
JOB START DATE               0
JOB END DATE                 0
USER ID                      0
OVER TIME HOURS              0
CHURN                        0
BUSINESS TRAVEL              0
ENVIRONMENT SATISFACTION     0
JOB SATISFACTION             0
MARTIAL STATUS               0
PERCENTAGE SALARY HIKE       0
PERFORMANCE RATING           0
RELATIONSHIP SATISFACTION    0
WORK LIFE BALANCE            0
dtype: int64


In [22]:
sf_df = table(df)

NameError: name 'table' is not defined

In [20]:
CATEGORICAL_COLUMNS = ["MAPPED ROLE CLEAN","SEX", "ETHNICITY","HOSPITAL TYPE", "HOSPITAL OWNERSHIP","COMPANY NAME","CITY","STATE","DISTANCE", 
                       "DEGREE CLEAN","BUSINESS TRAVEL","ENVIRONMENT SATISFACTION","JOB SATISFACTION","MARTIAL STATUS","PERFORMANCE RATING","RELATIONSHIP SATISFACTION","WORK LIFE BALANCE"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE MONTHS", "MONTHS AFTER COLLEGE", "BIRTH YEAR","AGE","USER ID", "OVER TIME HOURS", "PERCENTAGE SALARY HIKE",]
Date_COLUMNS = ['SCHOOL END DATE', 'JOB START DATE', 'JOB END DATE']
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]